In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import json
import re
from tqdm import tqdm
from pyspark.sql.functions import broadcast
from datetime import datetime, timedelta
import os
os.chdir('/home/y0c07y1/rpc_feature_extraction/rpc_model/')
from utils.utils import *
import gcsfs

In [2]:
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql import functions as func
import pyspark.sql as pss
from pyspark.sql.functions import col
from pyspark.sql import types as t
from pyspark.sql.types import (
    StringType,
    MapType,
    LongType,
    DoubleType,
    FloatType,
    StructField,
    StructType,
    IntegerType,
    ArrayType
)
#spark = pss.SparkSession.builder.appName("dctr_test3").config(conf=pyspark.SparkConf().setAll([('spark.yarn.queue', 'root.critical')])).enableHiveSupport().getOrCreate()
def gen_spark_session(ds):
    ds_nodash = ds.replace("-", "")

    return pyspark.sql.SparkSession.builder.config(
            "hive.exec.dynamic.partition", True,
        ).config(
            "hive.exec.dynamic.partition.mode", "nonstrict"
        ).config('mapreduce.input.fileinputformat.input.dir.recursive',True
        ).config('spark.hive.mapred.supports.subdirectories',True
        ).config("spark.sql.crossJoin.enabled", "true"
        ).config("spark.sql.broadcastTimeout", "36000"
        ).config("spark.sql.sources.partitionOverwriteMode","dynamic"
        ).config("spark.hadoop.orc.overwrite.output.file",True
        ).config("spark.yarn.executor.memoryOverhead", '20g'
        ).config("spark.executor.memory",'12g'
        ).config("spark.driver.memory",'12g'
        ).config("spark.dynamicAllocation.enabled",'true'
        ).config("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation",'true'
        ).config("spark.sql.hive.convertMetastoreOrc", 'false'
        ).appName(APP_NAME % ds_nodash).enableHiveSupport().getOrCreate()


APP_NAME = 'rpc_feature_extractionjupyter_%s'
spark = gen_spark_session('2022-12/7')
spark.sparkContext.setLogLevel("ERROR")

# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10000000)

In [3]:
from __future__ import absolute_import

from pyspark.sql import functions as func
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, SparkSession
from pyspark.sql.functions import col, struct, to_json, udf, expr
from pyspark.sql.window import Window
from pyspark.sql.types import *

import argparse
from multiprocessing import Pool
import sys, os, json, shutil, glob, warnings, gc, random, collections, csv, re, logging, time, pickle
from pathlib import Path
from datetime import datetime
from google.cloud import storage

import pandas as pd
import numpy as np
import lightgbm as lgb
import gcsfs

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from utils.utils import *

/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [4]:
def wmae(y, y_pred, weights):
    error = np.abs(y - y_pred)*weights
    return np.sum(error)/np.sum(weights)
def wrmse(y, y_pred, weights):
    error = np.power(y - y_pred, 2)*weights
    return np.sqrt(np.sum(error)/np.sum(weights))
def train(df_trn, df_val, feature_cols, target_col, lgb_params, model_name, weight_col=None):
    # build dataset
    if weight_col:
        dtrain = lgb.Dataset(df_trn[feature_cols],
                             df_trn[target_col].values,
                             weight=df_trn[weight_col].values,
                             #weight=np.log(df_trn[weight_col].values + np.e),
                             free_raw_data=False,
                             silent=False,
                             )
        dvalid = lgb.Dataset(df_val[feature_cols],
                             df_val[target_col].values,
                             weight=df_val[weight_col].values,
                             #weight=np.log(df_val[weight_col].values + np.e),
                             free_raw_data=False,
                             silent=False,
                             )
    # train model
    model = lgb.train(lgb_params,
                      dtrain,
                      valid_sets=[dtrain, dvalid],
                      verbose_eval=100)

    local_file = f'dp_output/{model_name}.txt'
    model.save_model(local_file, num_iteration=model.best_iteration)


def eval(df, feature_cols, target_col, model_name, weight_col=None):
    try:
        model = lgb.Booster(model_file=(f'dp_output/{model_name}.txt'))
    except:
        raise RuntimeError(f'{model_name} does not exist...')
    y_pred = model.predict(df[feature_cols])
    y_pred = np.array([max(0,x) if np.isnan(x)==False else x for x in y_pred])
    WRMSE = wrmse(df[target_col].values, y_pred, df[weight_col].values)
    WMAE = wmae(df[target_col].values, y_pred, df[weight_col].values)
    return y_pred

lgb_params = {
'n_estimators': 4000,
'metric': 'rmse',
'learning_rate': 0.02,
'objective': 'regression',
'boosting_type': 'gbdt',
'subsample': 0.9,
'bagging_freq': 10,
'colsample_bytree': 0.33,
'verbose': -1,
'num_leaves': 63,
'max_depth': 10,
'seed': 42,
'n_jobs': -1,
'early_stopping_rounds': 200,
}
config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
adid_name = config["hash_id"]["adid"]
is_mobile_name = config["hash_id"]["is_mobile"]
source_id_name = config["hash_id"]["source_id"]

In [65]:
kk = []
for date in ['2023-01-28','2023-01-21','2023-01-14','2023-01-07','2022-12-31']:
    df1 = pd.read_csv('gs://msc_fair_airflow/rpc_model/2023-01-28/sample_data_full_feat/df_feat_all_coop_item_'+date+'.csv')
    for col in df1.columns:
        if '_l3' in col:
            df1.rename(columns = {col: col.replace('_l3','_l')}, inplace=True)
    df1 = df1[(df1['sem_rpc_brand_l'].isnull()==False)|(df1['sem_rpc_seller_l'].isnull()==False)]
    kk.append(df1)
df = pd.concat(kk)

/opt/conda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [93]:
df_p = df.copy()
df = df_p[df_p['sem_clicks_l']>=100/28]

In [95]:
len(df)

12732

In [67]:
# df = pd.read_csv('gs://msc_fair_airflow/rpc_model/2023-01-28/sample_data_full_feat_clickbase/df_feat_all_coop_item_2023-01-28.csv')

In [96]:

df_eval = df.copy()
# prepare train features
features = config["features"]
# clean features (all should be numerics)
features_set = ['uber_curr_item_price_w1', 'uber_avg_overall_rating_w1'] #hard coded for hot fix
df = convert_numeric(df, features_set)

# do train-val split
df['fold'] = 0
kf = KFold(5, shuffle=True, random_state=42)
for i, (_, val_idx) in enumerate(kf.split(df)):
    df['fold'].iloc[val_idx] = i
    
# filters
convrt_trn_brand_filter = (df[source_id_name]==2) & (df.sem_convrt_brand_l>=0) & (df.sem_convrt_brand_l<=0.3)
convrt_val_brand_filter = (df[source_id_name]==2) & (df.sem_convrt_brand_l>=0) & (df.sem_convrt_brand_l<=0.3)
#    ordersize_trn_filter = (df[source_id_name]==2) & (df.sem_orders_l>=3) & (df.sem_ordersize_l>=0) & (df.sem_ordersize_l<=500)
ordersize_trn_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
ordersize_val_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
rpc_trn_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)
rpc_val_brand_filter = (df[source_id_name]==2) & (df.sem_ordersize_brand_l>=0) & (df.sem_ordersize_brand_l<=500)

convrt_trn_seller_filter = (df[source_id_name]==2) & (df.sem_convrt_seller_l>=0) & (df.sem_convrt_seller_l<=0.3)
convrt_val_seller_filter = (df[source_id_name]==2) & (df.sem_convrt_seller_l>=0) & (df.sem_convrt_seller_l<=0.3)
#    ordersize_trn_filter = (df[source_id_name]==2) & (df.sem_orders_l>=3) & (df.sem_ordersize_l>=0) & (df.sem_ordersize_l<=500)
ordersize_trn_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
ordersize_val_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
rpc_trn_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
rpc_val_seller_filter = (df[source_id_name]==2) & (df.sem_ordersize_seller_l>=0) & (df.sem_ordersize_seller_l<=500)
filter_dic = {'convrt':{'brand':[convrt_trn_brand_filter, convrt_val_brand_filter], 'seller':[convrt_trn_seller_filter,convrt_val_seller_filter]},\
              'ordersize':{'brand':[ordersize_trn_brand_filter, ordersize_val_brand_filter], 'seller':[ordersize_trn_seller_filter,ordersize_val_seller_filter]},\
              'rpc':{'brand':[rpc_trn_brand_filter, rpc_val_brand_filter], 'seller':[rpc_trn_seller_filter,rpc_val_seller_filter]}}

df_eval['pred_convrt_brand'] = np.nan
df_eval['pred_ordersize_brand'] = np.nan
df_eval['pred_rpc_brand'] = np.nan
df_eval['pred_convrt_seller'] = np.nan
df_eval['pred_ordersize_seller'] = np.nan
df_eval['pred_rpc_seller'] = np.nan

targets = ['seller', 'brand']

In [97]:
config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
features = config["features"]
map_dic = {'w2':'m1','w1':'m2','v2':'m3','v3':'m4','v4':'m5'}
for model in ['convrt','ordersize']:
    for tar in ['seller','brand']:
        new_features = []
        for i in features[model][tar]:
            if not any(ext in i for ext in ['v5','v9','v12','v18']):
                kk = i
                if kk.startswith('sem_'):
                    for key in map_dic.keys():
                        kk = kk.replace(key,map_dic[key])
                new_features.append(kk)
                features[model][tar] = new_features

In [98]:
# config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
# features = config["features"]
# map_dic = {'w2':'c0','w1':'c1','v2':'c2','v3':'c3','v4':'c4'}
# for model in ['convrt','ordersize']:
#     for tar in ['seller','brand']:
#         new_features = []
#         for i in features[model][tar]:
#             if not any(ext in i for ext in ['v5','v9','v12','v18']):
#                 kk = i
#                 if kk.startswith('sem_'):
#                     for key in map_dic.keys():
#                         kk = kk.replace(key,map_dic[key])
#                 new_features.append(kk)
#                 features[model][tar] = new_features

In [99]:
for tar in targets:
    for fold in range(5):
        print('model'+str(fold))
        # convrt model
        model_features = features['convrt'][tar]
        target_col = "sem_convrt_"+tar+"_l"
        model_name = f'convrt_f{fold}'+'_'+tar
        trn_mask = filter_dic['convrt'][tar][0] & (df.fold != fold)
        df_trn = df[trn_mask]
        val_mask = filter_dic['convrt'][tar][1] & (df.fold == fold)
        df_val = df[val_mask]
        print('length convrt for ',tar,len(df_trn),len(df_val), len(df_trn)+len(df_val))
        train(df_trn, df_val, model_features, target_col, lgb_params, model_name, weight_col='sem_clicks_l')

        # Evaluate
        y_pred = eval(df_val, model_features, target_col, model_name, weight_col='sem_clicks_l')
        df_eval['pred_convrt_'+tar].loc[val_mask] = y_pred

        # order_size model
        model_features = features['ordersize'][tar]
        target_col = "sem_ordersize_"+tar+"_l"
        model_name = f'ordersize_f{fold}'+'_'+tar
        trn_mask = filter_dic['ordersize'][tar][0] & (df.fold != fold)
        df_trn = df[trn_mask]
        val_mask = filter_dic['ordersize'][tar][1] & (df.fold == fold)
        df_val = df[val_mask]
        print('length ordersize for ',tar,len(df_trn),len(df_val), len(df_trn)+len(df_val))
        train(df_trn, df_val, model_features, target_col, lgb_params, model_name, weight_col='sem_orders_'+tar+'_l')

        # Evaluate
        y_pred = eval(df_val, model_features, target_col, model_name, weight_col='sem_orders_'+tar+'_l')
        df_eval['pred_ordersize_'+tar].loc[val_mask] = y_pred

model0
length convrt for  seller 6226 1576 7802
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 0.0446286	valid_1's rmse: 0.0597979
[200]	training's rmse: 0.0352219	valid_1's rmse: 0.0570432
[300]	training's rmse: 0.0308737	valid_1's rmse: 0.0564377
[400]	training's rmse: 0.0277847	valid_1's rmse: 0.0562861
[500]	training's rmse: 0.025187	valid_1's rmse: 0.0562614
[600]	training's rmse: 0.0231277	valid_1's rmse: 0.0561727
[700]	training's rmse: 0.0211946	valid_1's rmse: 0.0562306
Early stopping, best iteration is:
[570]	training's rmse: 0.0236334	valid_1's rmse: 0.0561606
length ordersize for  seller 8646 2125 10771
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 3.61068	valid_1's rmse: 4.53374
[200]	training's rmse: 3.1588	valid_1's rmse: 4.40057
[300]	training's rmse: 2.9298	valid_1's rmse: 4.35895
[400]	training's rmse: 2.78701	valid_1's rmse: 4.34434
[500]	training's rmse: 2.67966	valid_1's rmse: 4.33598
[600

In [100]:
for tar in targets:
    df_eval['pred_rpc_'+tar] = df_eval['pred_convrt_'+tar] * df_eval['pred_ordersize_'+tar]
    target_cols = ['sem_convrt_'+tar+'_l', 'sem_ordersize_'+tar+'_l', 'sem_rpc_'+tar+'_l']
    pred_cols = ['pred_convrt_'+tar, 'pred_ordersize_'+tar, 'pred_rpc_'+tar]
    weight_cols = ['sem_clicks_l', 'sem_orders_'+tar+'_l', 'sem_clicks_l']
    for target_col, pred_col, weight_col in zip(target_cols, pred_cols, weight_cols):
        filter_NoNA = df_eval[target_col].notnull() & df_eval[pred_col].notnull() & df_eval[weight_col].notnull()
        WRMSE = wrmse(df_eval[target_col].loc[filter_NoNA].values, df_eval[pred_col].loc[filter_NoNA].values, df_eval[weight_col].loc[filter_NoNA].values)
        WMAE = wmae(df_eval[target_col].loc[filter_NoNA].values, df_eval[pred_col].loc[filter_NoNA].values, df_eval[weight_col].loc[filter_NoNA].values)
        print(f"{'_'.join(pred_col.split('_')[1:-1])} oof based on lightgbm WRMSE: {WRMSE: .5f}")
        print(f"{'_'.join(pred_col.split('_')[1:-1])} oof based on lightgbm WMAE: {WMAE: .5f}")

convrt oof based on lightgbm WRMSE:  0.05409
convrt oof based on lightgbm WMAE:  0.03542
ordersize oof based on lightgbm WRMSE:  4.08192
ordersize oof based on lightgbm WMAE:  1.58922
rpc oof based on lightgbm WRMSE:  0.90634
rpc oof based on lightgbm WMAE:  0.47463
convrt oof based on lightgbm WRMSE:  0.00677
convrt oof based on lightgbm WMAE:  0.00323
ordersize oof based on lightgbm WRMSE:  25.12017
ordersize oof based on lightgbm WMAE:  10.77429
rpc oof based on lightgbm WRMSE:  0.59016
rpc oof based on lightgbm WMAE:  0.25510


# deep learning

In [3]:
from deepctr_torch.models import DIEN
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [4]:
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM

In [9]:
data = pd.read_csv('gs://msc_fair_airflow/rpc_model/sample_data_full_feat/df_feat_all_coop_item_2023-01-16.csv',nrows = 10000)
sparse_features = ['uber_dept_w1',
 'uber_subcat_w1',
 'uber_super_dept_w1',
 'uber_division_w1',
 'uber_cat_w1']
target = ['sem_rpc_seller_l']

In [17]:
config = load_json('gs://msc_fair_airflow/rpc_model/config_rpc.json')
adid_name = config["hash_id"]["adid"]
is_mobile_name = config["hash_id"]["is_mobile"]
source_id_name = config["hash_id"]["source_id"]
features = config['features']
model_features = features['convrt']['seller']

In [ ]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [24]:
[i for i in list(set(data.columns)-set(model_features)) if '_brand_' not in i and '_l' not in i]

['uber_dept_w1',
 'uber_subcat_w1',
 'catalog_item_id',
 'source_id',
 'seller_id',
 'adid',
 'uber_super_dept_w1',
 'uber_division_w1',
 'uber_cat_w1']

In [5]:
data = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zip"]
target = ['rating']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

In [7]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
model.compile("adam", "mae", metrics=['htdhtr'], )

history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2,
                    validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  3.5633
Epoch 2/10
0s - loss:  3.5127
Epoch 3/10
0s - loss:  3.4692
Epoch 4/10
0s - loss:  3.4275
Epoch 5/10
0s - loss:  3.3875
Epoch 6/10
0s - loss:  3.3485
Epoch 7/10
0s - loss:  3.3075
Epoch 8/10
0s - loss:  3.2645
Epoch 9/10
0s - loss:  3.2192
Epoch 10/10
0s - loss:  3.1717


In [26]:
train

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
92,76,49,3,974658834,"20,000 Leagues Under the Sea (1954)",Adventure|Children's|Fantasy|Sci-Fi,1,5,15,15
184,71,3,5,974697744,Dead Man Walking (1995),Drama,1,6,12,163
11,7,173,5,976316283,High Fidelity (2000),Comedy,1,2,2,140
27,73,100,4,974677761,On the Waterfront (1954),Crime|Drama,0,6,12,9
30,65,122,2,974699637,"Player, The (1992)",Comedy|Drama,1,1,0,162
...,...,...,...,...,...,...,...,...,...,...
22,175,137,3,960328279,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,1,4,7,3
187,67,186,5,974753321,Meet the Parents (2000),Comedy,1,2,7,151
42,143,101,4,965094644,"Last Emperor, The (1987)",Drama|War,1,5,0,1
141,171,80,4,961055887,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi,1,2,11,161


In [21]:
pd.read_csv("./movielens_sample.txt")['user_id'].nunique()

193